# snippet generator

> Tool for user's snippets generation


In [ ]:
# | default_exp tools.jupyter.snippet_generator

In [ ]:
# | export
# | hide

import argparse
import json
import shlex

from IPython.core.magic import register_cell_magic

In [ ]:
# | export
# | hide


def _convert_to_snippet(
    code: str, snippet_name: str, prefix: str, description: str | None = None
):
    if not snippet_name or not prefix:
        raise ValueError(
            "snippet_name and prefix must be provided, use -n and -d arguments"
        )
    lines = code.split("\n")[:-1]
    json_data = {
        snippet_name: {
            "prefix": prefix,
            "body": lines,
            "description": description if description else snippet_name,
        }
    }
    str_repr = json.dumps(json_data, indent=4)
    return "\n".join(str_repr.split("\n")[1:-1])

In [ ]:
# | export


@register_cell_magic
def to_snippet_str(line, cell):
    """Convert a cell to a VSCode snippet string

    Use it as a cell magic in Jupyter notebooks:
    `%%to_snippet_str -n snippet_name -p prefix -d description`
    -d is optional

    -n is the name of the snippet in snippets declaration file
    -p is the prefix for the snippet, it is used to trigger the snippet, e.g. if the prefix is "snippet", then typing "snippet" in
        the editor will trigger the snippet (it will be injected into autocompletion)
    -d is the description of the snippet, it is shown in the autocompletion

    Parameters
    ----------
    line : int, it will be specified by Jupyter
    cell : int, it will be specified by Jupyter
    """
    # Use shlex.split for a shell-like split keeping quoted strings together
    tokens = shlex.split(line)

    # Initialize argparse parser
    parser = argparse.ArgumentParser()
    parser.add_argument("-n", "--name", help="Snippet name")
    parser.add_argument("-p", "--prefix", help="Prefix for the snippet")
    parser.add_argument("-d", "--description", help="Description for the snippet")
    parser.add_argument(
        "-f",
        "--file",
        help="File to save the snippet to",
        default="~/Library/Application Support/Code/User/snippets/jupyter_notebook_snippets.code-snippets",
    )

    # Use shlex.split to treat the line as a command-line input
    args = parser.parse_args(tokens)

    snippet = _convert_to_snippet(cell, args.name, args.prefix, args.description)
    print(snippet)

#### Snippets generation

Use `%%to_snippet_str` magic function to choose a cell for snippet export. Exporting snippet is used for generation of snippets string that may be copied to the clipboard and pasted into the snippets declaration file.

In VSCode this file may be found or created at the path specified in the `snippets` setting.

1. Open the Command Palette by pressing `Ctrl+Shift+P` (Windows/Linux) or `Cmd+Shift+P` (macOS).
2. Type "Preferences: Configure User Snippets" and select the language for which you want to create a snippet (e.g., "Python").
3. Select the "New Global Snippets File" option and save the file with a descriptive name, such as "python-snippets.json".
4. In the new snippets file, add a new snippet object with a descriptive name and a prefix that you will use to trigger the snippet (e.g., "json_decoder").
5. Set the "body" property of the snippet object to the code that you pasted from the Jupyter Notebook snippet.
6. Save the snippets file.

You can now use your new snippet by typing the prefix that you set and pressing `Tab`. The code from the Jupyter Notebook snippet will be inserted into your code file.


We could export following code to a string representation


In [ ]:
%%to_snippet_str -n test_a -p test_a
#| hide

def a():
    pass

    "test_a": {
        "prefix": "test_a",
        "body": [
            "#| code_fold: show",
            "",
            "def a():",
            "    pass"
        ],
        "description": "test_a"
    }


And copy it to snippets file


In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()